In [ ]:
import json
import os
from datetime import datetime
import pandas as pd
import numpy as np

def read_json(file_): 
    with open(file_,'r') as file: #wczytywanie json funkcja open()
        return json.load(file)

def get_df(data,datetime_str):
    df = pd.DataFrame(data['data'])
    df = pd.concat([df.drop(["status"],axis=1),pd.json_normalize(df["status"])],axis=1)
    df.rename(columns={"ts":"ts_status"},inplace=True)
    df['time'] = pd.to_datetime(datetime_str,format="%y%m%d_%H%M")
    df_prices = df[['point_id','time','prices']].explode(["prices"]).reset_index(drop=True)
    df_prices = pd.concat([df_prices, pd.json_normalize(df_prices["prices"])], axis = 1)
    return df.drop('prices',axis=1), df_prices.drop('prices',axis=1)

if __name__=="__main__":
    #definicja ścieżki z outputu z 2 zmiennymi (output + filename)
    output_directory = "./output"
    csv_file_path_data = os.path.join(output_directory, "data_all_days.csv")
    csv_file_path_prices = os.path.join(output_directory, "prcies_all_days.csv")
    time_start = '2024-11-22 09:58:00'
    
    #definiujemy input
    folder_path =r"C:\Users\bekert\OneDrive - KPMG\Alteryx\17.Mail_zlecenia\241122_Pobierania_jsona_przez_7\Data"
    df = pd.DataFrame()
    df_prices = pd.DataFrame()
    #sprawdzamy wszytskie pliki z folder patha i je zaczytujemy
    for file_name in os.listdir(folder_path):
        data = read_json(os.path.join(folder_path,file_name)) #szczytaneie pliku z listy i dodanie file_name
        if df.empty:
            df, df_prices = get_df(data,file_name[:11])
        else:
            df_temp, df_prices_temp = get_df(data,file_name[:11])
            df = pd.concat([df,df_temp],ignore_index=True)
            df_prices = pd.concat([df_prices,df_prices_temp],ignore_index=True)
    
    df.loc[(df['time']> pd.to_datetime(time_start))].to_csv(csv_file_path_data,sep=";", index=False)
    df_prices.loc[(df_prices['time']> pd.to_datetime(time_start))].to_csv(csv_file_path_prices,sep=";", index=False)

,point_id,code,max_output_unit,availability,status,ts_status,time
0,13480,PL-GJC-EEVP01005,NaN,1.0,1.0,2020-11-18T14:19:26+01:00,2024-11-22 08:11:00
1,13481,PL-GJC-EEVP01008,NaN,1.0,1.0,2020-11-18T17:50:26+01:00,2024-11-22 08:11:00
2,13493,PL-GJC-EEVP01018,NaN,1.0,1.0,2023-06-09T17:38:36+02:00,2024-11-22 08:11:00
3,13494,PL-GJC-EEVP01019,NaN,1.0,0.0,2023-06-09T17:42:43+02:00,2024-11-22 08:11:00
4,13501,PL-GJC-EEVP01027,NaN,1.0,1.0,2023-06-08T11:38:35+02:00,2024-11-22 08:11:00
...,...,...,...,...,...,...,...
16781,312390,PL-BYN-EBFE72A5E,NaN,1.0,1.0,2024-11-21T21:44:02+01:00,2024-11-22 08:16:00
16782,312651,PL-BYN-E8345E13E,NaN,1.0,1.0,2024-11-22T00:27:53+01:00,2024-11-22 08:16:00
16783,312652,PL-BYN-E3AE9287D,NaN,1.0,1.0,2024-11-22T01:47:38+01:00,2024-11-22 08:16:00
16784,312653,PL-BYN-E5D9216B7,NaN,1.0,1.0,2024-11-22T08:10:12+01:00,2024-11-22 08:16:00
